In [1]:
# import modules

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import pandas as pd
import seaborn as sns

In [2]:
# import GEO reference data

# import table with accession, platform, and series
geoAPS = pd.read_csv('../geo_sample.csv')

# import table with datasets IDs
geoDS = pd.read_csv('../geo_series_table.csv', low_memory = False)
geoDS.columns = ['Series', 'Title', 'Series Type', 'Taxonomy', 
                 'Sample Count', 'Datasets', 'Supplementary Types', 
                 'Supplementary Links', 'PubMed ID', 'SRA Accession', 
                 'Contact', 'Release Date']

# add datasets column by merging
allData = pd.merge(geoAPS, geoDS, how = 'outer', on = 'Series')
geoReference = allData[['Series', 'Accession', 'Platform', 'Datasets']]

geoReference

,Series,Accession,Platform,Datasets
0,GSE506,GSM1,GPL4,NaN
1,GSE506,GSM2,GPL4,NaN
2,GSE462,GSM3,GPL5,NaN
3,GSE462,GSM4,GPL5,NaN
4,GSE462,GSM5,GPL5,NaN
...,...,...,...,...
3280011,GSE136775,NaN,NaN,NaN
3280012,GSE136776,NaN,NaN,NaN
3280013,GSE137458,NaN,NaN,NaN
3280014,GSE137562,NaN,NaN,NaN


In [3]:
# import SRA reference data
sraReference = pd.read_csv('../sraIDfull.csv', error_bad_lines = False, low_memory=False, quoting=3)
sraReference = sraReference[['SRAStudy', 'Run', 'Experiment', 
                             'BioProject', 'Submission', 'Sample']]
sraReference

,SRAStudy,Run,Experiment,BioProject,Submission,Sample
0,ERP000767,ERR3550121,ERX3556726,PRJEB2600,ERA2143351,ERS3773255
1,ERP000767,ERR3550142,ERX3556747,PRJEB2600,ERA2143351,ERS3773255
2,ERP000767,ERR3550147,ERX3556752,PRJEB2600,ERA2143351,ERS3773263
3,ERP000767,ERR3550125,ERX3556730,PRJEB2600,ERA2143351,ERS3773263
4,ERP000767,ERR3550137,ERX3556742,PRJEB2600,ERA2143351,ERS3773252
...,...,...,...,...,...,...
7676058,SRP002493,SRR049674,SRX020916,NaN,SRA017695,SRS073299
7676059,SRP002493,SRR049675,SRX021028,NaN,SRA017695,SRS073299
7676060,SRP002493,SRR049677,SRX021029,NaN,SRA017695,SRS073299
7676061,SRP002064,SRR042496,SRX020084,PRJNA46359,DIEGO,SRS011854


In [4]:
# import data scraped from PubMed XML files
pmcData = pd.read_csv('../data_tables/preFilterMatrix.csv')
pmcData

# temporary mini dataframe...
# pmcData = pmcData.iloc[141499:144406]
# pmcData

,journal,pmc_ID,accession
0,Alzheimers_Res_Ther,PMC3707052,GSM1
1,Alzheimers_Res_Ther,PMC3706879,GSE45534
2,Alzheimers_Res_Ther,PMC3706879,GSE45534
3,Alzheimers_Res_Ther,PMC3706879,GSE45534
4,Alzheimers_Res_Ther,PMC3706879,GSE45534
...,...,...,...
144401,Dis_Markers,PMC3834650,GDS1059
144402,Dis_Markers,PMC3834650,GDS1059
144403,Dis_Markers,PMC3834650,GDS1
144404,Case_Rep_Hematol,PMC6462343,PRJNA437812


In [5]:
# define functions returning corresponding SRA study or GEO series
def corrSRAStudy(acc, col):
    indices = sraReference.index[sraReference[col] == acc].tolist()
    if(len(indices) > 0):
        allSRAStudies = sraReference.loc[indices][['SRAStudy']]
        modeSRA = allSRAStudies.mode()
        if(modeSRA.empty):
            return 'NaN'
        else:
            return modeSRA.iloc[0,0]
    else:
        return 'NaN'

def corrGEOSeries(acc, col):
    indices = geoReference.index[geoReference[col] == acc].tolist()
    if(len(indices) > 0):
        allGEOSeries = geoReference.loc[indices][['Series']]
        modeGEO = allGEOSeries.mode()
        if(modeGEO.empty):
            return 'NaN'
        else:
            return modeGEO.iloc[0,0]
    else:
        return 'NaN'
    
# define functions performing QC on accessions, checking if they exist in the references
def sraChecksOut(acc, col):
    if(acc in sraReference[col].tolist()):
        return True
    else:
        return False
    
def geoChecksOut(acc, col):
    if(acc in geoReference[col].tolist()):
        return True
    else:
        return False

In [6]:
# check every accession against the corresponding reference, and convert to STUDY/SERIES
# add converted accessions to a new list, to be added as a column later

converted_acc = []

for acc in pmcData['accession']:
    style = ''
    # assign a style to the accession, corresponds to column names
    # SRA styles
    if('SRP' in acc or 'ERP' in acc or 'DRP' in acc):
        style = 'SRAStudy'
    elif('SRR' in acc or 'ERR' in acc or 'DRR' in acc):
        style = 'Run'
    elif('SRX' in acc or 'ERX' in acc or 'DRX' in acc):
        style = 'Experiment'
    elif('PRJNA' in acc or 'PRJD' in acc or 'PRJEB' in acc):
        style = 'BioProject'
    elif('SRA' in acc or 'ERA' in acc or 'DRA' in acc):
        style = 'Submission'
    elif('SRS' in acc or 'ERS' in acc or 'DRS' in acc):
        style = 'Sample'
    # GEO styles
    elif('GSE' in acc):
        style = 'Series'
    elif('GSM' in acc):
        style = 'Accession'
    elif('GPL' in acc):
        style = 'Platform'
    elif('GDS' in acc):
        style = 'Datasets'
    else:
        style = ''
        
    # for SRA accessions, check if they exist in the SRA reference
    # ...and add corresponding STUDY ID to new column
    if(style == 'SRAStudy' or style == 'Run' or style == 'Experiment' or 
       style == 'BioProject' or style == 'Submission' or style == 'Sample'):
        if(not sraChecksOut(acc, style)):
            converted_acc.append('NaN')
        else:
            converted_acc.append(corrSRAStudy(acc, style))
    # for GEO accessions, check if they exist in the GEO reference
    # ...and add corresponding SERIES ID to new column
    elif(style == 'Series' or style == 'Accession' or style == 'Platform' or style == 'Datasets'):
        if(not geoChecksOut(acc, style)):
            converted_acc.append('NaN')
        else:
            converted_acc.append(corrGEOSeries(acc, style))
    # something doesn't belong, record as missing
    else:
        converted_acc.append('NaN')

# add the converted accession list as a new column in PubMed data table
pd.set_option('display.min_rows', 50)
pmcData['converted_accession'] = converted_acc
pmcData = pmcData.drop_duplicates()
pmcData

,journal,pmc_ID,accession,converted_accession
0,Alzheimers_Res_Ther,PMC3707052,GSM1,GSE506
1,Alzheimers_Res_Ther,PMC3706879,GSE45534,GSE45534
5,Alzheimers_Res_Ther,PMC4255636,GSE5281,GSE5281
6,Alzheimers_Res_Ther,PMC4731966,GSE67036,GSE67036
7,Alzheimers_Res_Ther,PMC4731966,GSE1297,GSE1297
8,Alzheimers_Res_Ther,PMC4731966,GSE48350,GSE48350
9,Alzheimers_Res_Ther,PMC4731966,GSE36980,GSE36980
10,Alzheimers_Res_Ther,PMC4731966,GSE12679,GSE12679
11,Alzheimers_Res_Ther,PMC4731966,GSE12293,GSE12293
17,Alzheimers_Res_Ther,PMC4731966,GSE46871,GSE46871


In [9]:
# print(len(converted_acc))
# print(len(pmcData['pmc_ID'].tolist()))

# pmcData.iloc[141499:141505]

#corrSRAStudy('ERR3293975', 'Run')

# indices = sraReference.index[sraReference['Run'] == 'ERR3293975'].tolist()
# indices

# allSRAStudies = sraReference.loc[indices][['SRAStudy']]
# allSRAStudies

# allSRAStudies.mode().empty

# print(allSRAStudies.empty)

# sraReference.iloc[48047]

144406

In [8]:
# save to CSV file
pmcData.to_csv('postFilterMatrix.csv')